<h3>Remembering Python...</h3>

Python boots up with __builtins__ already in the namespace and checked as a part of the name resolution protocol...

In [3]:

# I, Python am built from types, such as builtin types:

the_builtins = dir(__builtins__) # always here

print(the_builtins)  # no need to import

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError', 'SystemExit', 'TabEr

Lets check our understanding that the native types we count on to build more complex types, live in __builtins__:

In [4]:
for the_string in ["Decimal", "list", "tuple", "dict", "int", "Fraction", "float"]:
    if the_string in the_builtins:
        print("Yes I am a native type: ", the_string)
        assert type(eval(the_string)) == type # all types in this club
    else:
        print("No, I'm not native: ", the_string)

# usually up top
from string import ascii_lowercase as all_lowers
from random import shuffle

No, I'm not native:  Decimal
Yes I am a native type:  list
Yes I am a native type:  tuple
Yes I am a native type:  dict
Yes I am a native type:  int
No, I'm not native:  Fraction
Yes I am a native type:  float


And now for something completely different, lets define a class that does substitution based on a permutation of lower-case ascii letters plus space.  Such a type is given more substantial implementation in the form of our Px class, which allows permutations to multiply, giving more permuations.

In [6]:
# usually up top
from string import ascii_lowercase as all_lowers
from random import shuffle

class P:
    """
    class Px is the more sophisticated version of this class
    """
    def __init__(self, p=None):
        if not p:
            original = all_lowers + ' '
            scrambled = list(original)
            shuffle(scrambled)            
            self.perm = dict(zip(original, scrambled))
        else:
            self.perm = p
        
    def __invert__(self):
        """reverse my perm, make a new me"""
        reverse = dict(zip(self.perm.values(), self.perm.keys()))
        return P(reverse)  # <-- new P instance
        
    def encrypt(self, s):
        output = ""
        for c in s:
            output += self.perm[c]
        return output
            
    def decrypt(self, s):
        rev = ~self  # <-- new P instance
        return rev.encrypt(s) # <-- symmetric key

     
p = P()
m = "able was i ere i saw elba"
c = p.encrypt(m)
print(m)  # plaintext
print(c)  # ciphertext
d = p.decrypt(c)
print(d)

able was i ere i saw elba
hspkyghuylykikylyuhgykpsh
able was i ere i saw elba
